# شبکه‌های مولد

شبکه‌های عصبی بازگشتی (RNNs) و انواع سلول‌های دروازه‌دار آن‌ها مانند سلول‌های حافظه کوتاه‌مدت بلند (LSTMs) و واحدهای بازگشتی دروازه‌دار (GRUs) مکانیزمی برای مدل‌سازی زبان فراهم کردند، یعنی آن‌ها می‌توانند ترتیب کلمات را یاد بگیرند و پیش‌بینی‌هایی برای کلمه بعدی در یک دنباله ارائه دهند. این قابلیت به ما اجازه می‌دهد تا از RNNها برای **وظایف مولد** استفاده کنیم، مانند تولید متن معمولی، ترجمه ماشینی، و حتی توضیح‌نویسی تصاویر.

در معماری RNN که در واحد قبلی مورد بحث قرار گرفت، هر واحد RNN حالت مخفی بعدی را به عنوان خروجی تولید می‌کرد. با این حال، ما می‌توانیم خروجی دیگری به هر واحد بازگشتی اضافه کنیم که به ما اجازه می‌دهد یک **دنباله** (که طول آن برابر با دنباله اصلی است) تولید کنیم. علاوه بر این، می‌توانیم از واحدهای RNN استفاده کنیم که در هر مرحله ورودی دریافت نمی‌کنند و فقط یک بردار حالت اولیه می‌گیرند و سپس یک دنباله از خروجی‌ها تولید می‌کنند.

در این دفترچه، ما بر مدل‌های مولد ساده تمرکز خواهیم کرد که به ما کمک می‌کنند متن تولید کنیم. برای سادگی، بیایید یک **شبکه سطح کاراکتر** بسازیم که متن را حرف به حرف تولید کند. در طول آموزش، باید یک متن مرجع را بگیریم و آن را به دنباله‌های حرف تقسیم کنیم.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## ساخت واژگان کاراکتر

برای ساخت شبکه تولیدی در سطح کاراکتر، باید متن را به جای کلمات به کاراکترهای جداگانه تقسیم کنیم. این کار با تعریف یک توکنایزر متفاوت قابل انجام است:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


بیایید مثالی ببینیم از اینکه چگونه می‌توانیم متن را از مجموعه داده خود رمزگذاری کنیم:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## آموزش یک RNN مولد

روش آموزش RNN برای تولید متن به این صورت است. در هر مرحله، یک دنباله از کاراکترها به طول `nchars` را می‌گیریم و از شبکه می‌خواهیم که برای هر کاراکتر ورودی، کاراکتر خروجی بعدی را تولید کند:

![تصویری که نمونه‌ای از تولید کلمه 'HELLO' توسط RNN را نشان می‌دهد.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

بسته به سناریوی واقعی، ممکن است بخواهیم برخی کاراکترهای خاص مانند *پایان دنباله* `<eos>` را نیز شامل کنیم. در مورد ما، هدف فقط آموزش شبکه برای تولید متن بی‌پایان است، بنابراین اندازه هر دنباله را برابر با `nchars` تنظیم می‌کنیم. در نتیجه، هر مثال آموزشی شامل `nchars` ورودی و `nchars` خروجی خواهد بود (که دنباله ورودی یک نماد به سمت چپ جابه‌جا شده است). یک مینی‌بچ شامل چندین دنباله از این نوع خواهد بود.

روش تولید مینی‌بچ‌ها این است که هر متن خبری به طول `l` را بگیریم و تمام ترکیب‌های ممکن ورودی-خروجی را از آن تولید کنیم (تعداد این ترکیب‌ها برابر با `l-nchars` خواهد بود). این ترکیب‌ها یک مینی‌بچ را تشکیل می‌دهند و اندازه مینی‌بچ‌ها در هر مرحله آموزشی متفاوت خواهد بود.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

حالا بیایید شبکه تولیدکننده را تعریف کنیم. این شبکه می‌تواند بر اساس هر سلول بازگشتی که در واحد قبلی بررسی کردیم (ساده، LSTM یا GRU) باشد. در مثال ما از LSTM استفاده خواهیم کرد.

از آنجا که شبکه کاراکترها را به عنوان ورودی دریافت می‌کند و اندازه واژگان نسبتاً کوچک است، نیازی به لایه تعبیه نیست؛ ورودی کدگذاری‌شده به صورت یک‌داغ می‌تواند مستقیماً به سلول LSTM ارسال شود. با این حال، چون ما شماره‌های کاراکترها را به عنوان ورودی ارسال می‌کنیم، باید قبل از ارسال به LSTM آنها را به صورت یک‌داغ کدگذاری کنیم. این کار با فراخوانی تابع `one_hot` در طول عبور `forward` انجام می‌شود. رمزگذار خروجی یک لایه خطی خواهد بود که حالت مخفی را به خروجی کدگذاری‌شده به صورت یک‌داغ تبدیل می‌کند.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

در طول آموزش، ما می‌خواهیم بتوانیم متن تولید شده را نمونه‌برداری کنیم. برای این کار، تابعی به نام `generate` تعریف می‌کنیم که رشته‌ای به طول `size` تولید می‌کند و از رشته اولیه `start` شروع می‌شود.

روش کار به این صورت است: ابتدا، کل رشته `start` را از طریق شبکه عبور می‌دهیم و حالت خروجی `s` و کاراکتر پیش‌بینی‌شده بعدی `out` را دریافت می‌کنیم. از آنجا که `out` به صورت یک‌کدگذاری یک‌داغ است، از `argmax` استفاده می‌کنیم تا شاخص کاراکتر `nc` را در واژه‌نامه پیدا کنیم و با استفاده از `itos` کاراکتر واقعی را مشخص کرده و به لیست کاراکترهای نتیجه `chars` اضافه کنیم. این فرآیند تولید یک کاراکتر، به تعداد `size` بار تکرار می‌شود تا تعداد کاراکترهای مورد نیاز تولید شود.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

حالا بیایید آموزش را شروع کنیم! حلقه آموزش تقریباً مشابه تمام مثال‌های قبلی ما است، اما به جای دقت، متن تولید شده نمونه‌ای را هر ۱۰۰۰ دوره چاپ می‌کنیم.

توجه ویژه‌ای باید به نحوه محاسبه خطا شود. ما باید خطا را با توجه به خروجی یک‌داغ‌شده `out` و متن مورد انتظار `text_out` که لیستی از شاخص‌های کاراکتر است، محاسبه کنیم. خوشبختانه، تابع `cross_entropy` خروجی شبکه غیرنرمال‌شده را به عنوان آرگومان اول و شماره کلاس را به عنوان آرگومان دوم انتظار دارد، که دقیقاً همان چیزی است که داریم. این تابع همچنین میانگین‌گیری خودکار بر اساس اندازه مینی‌بچ را انجام می‌دهد.

ما همچنین آموزش را با محدود کردن به تعداد نمونه‌های `samples_to_train` محدود می‌کنیم تا زمان زیادی منتظر نمانیم. شما را تشویق می‌کنیم که آزمایش کنید و آموزش طولانی‌تر را امتحان کنید، شاید برای چندین دوره (که در این صورت نیاز دارید یک حلقه دیگر در اطراف این کد ایجاد کنید).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

این مثال از قبل متن نسبتاً خوبی تولید می‌کند، اما می‌توان آن را به چندین روش بهبود داد:

* **بهبود تولید مینی‌بچ‌ها**. روشی که برای آماده‌سازی داده‌ها برای آموزش استفاده کردیم، تولید یک مینی‌بچ از یک نمونه بود. این روش ایده‌آل نیست، زیرا اندازه مینی‌بچ‌ها متفاوت است و برخی از آن‌ها حتی قابل تولید نیستند، چون متن کوچک‌تر از `nchars` است. همچنین، مینی‌بچ‌های کوچک به اندازه کافی از GPU استفاده نمی‌کنند. بهتر است یک بخش بزرگ از متن را از تمام نمونه‌ها بگیریم، سپس تمام جفت‌های ورودی-خروجی را تولید کنیم، آن‌ها را به هم بزنیم و مینی‌بچ‌هایی با اندازه برابر تولید کنیم.

* **LSTM چندلایه**. منطقی است که 2 یا 3 لایه از سلول‌های LSTM را امتحان کنیم. همان‌طور که در واحد قبلی اشاره کردیم، هر لایه از LSTM الگوهای خاصی را از متن استخراج می‌کند، و در مورد تولیدکننده سطح کاراکتر می‌توان انتظار داشت که لایه‌های پایین‌تر LSTM مسئول استخراج هجاها باشند و لایه‌های بالاتر - کلمات و ترکیب‌های کلمات. این کار به سادگی با ارسال پارامتر تعداد لایه‌ها به سازنده LSTM قابل اجرا است.

* همچنین ممکن است بخواهید با **واحدهای GRU** آزمایش کنید و ببینید کدام یک عملکرد بهتری دارند، و با **اندازه‌های مختلف لایه‌های مخفی** نیز آزمایش کنید. لایه مخفی بیش از حد بزرگ ممکن است منجر به بیش‌برازش شود (مثلاً شبکه متن دقیق را یاد می‌گیرد)، و اندازه کوچک‌تر ممکن است نتیجه خوبی تولید نکند.


## تولید متن نرم و دما

در تعریف قبلی `generate`، ما همیشه کاراکتری را که بالاترین احتمال را داشت به عنوان کاراکتر بعدی در متن تولید شده انتخاب می‌کردیم. این باعث می‌شد که متن اغلب بین توالی‌های کاراکتری مشابه بارها و بارها "چرخه" بزند، مانند این مثال:
```
today of the second the company and a second the company ...
```

با این حال، اگر به توزیع احتمالات برای کاراکتر بعدی نگاه کنیم، ممکن است تفاوت بین چند احتمال بالاتر خیلی زیاد نباشد، به عنوان مثال، یک کاراکتر می‌تواند احتمال ۰.۲ داشته باشد و دیگری ۰.۱۹ و غیره. برای مثال، وقتی به دنبال کاراکتر بعدی در توالی '*play*' هستیم، کاراکتر بعدی می‌تواند به همان اندازه احتمالاً یک فاصله باشد یا **e** (مانند کلمه *player*).

این ما را به این نتیجه می‌رساند که همیشه "منصفانه" نیست که کاراکتری با احتمال بالاتر را انتخاب کنیم، زیرا انتخاب کاراکتر دوم با احتمال بالاتر نیز ممکن است به متنی معنادار منجر شود. عاقلانه‌تر این است که **نمونه‌گیری** از کاراکترها را بر اساس توزیع احتمالاتی که توسط خروجی شبکه داده شده انجام دهیم.

این نمونه‌گیری می‌تواند با استفاده از تابع `multinomial` که توزیع موسوم به **توزیع چندجمله‌ای** را پیاده‌سازی می‌کند، انجام شود. تابعی که این تولید متن **نرم** را پیاده‌سازی می‌کند در زیر تعریف شده است:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

ما یک پارامتر دیگر به نام **دما** معرفی کرده‌ایم که برای نشان دادن میزان پایبندی به بالاترین احتمال استفاده می‌شود. اگر دما ۱.۰ باشد، نمونه‌گیری چندجمله‌ای منصفانه انجام می‌دهیم، و زمانی که دما به بی‌نهایت می‌رسد - همه احتمالات برابر می‌شوند و ما به صورت تصادفی کاراکتر بعدی را انتخاب می‌کنیم. در مثال زیر می‌توان مشاهده کرد که متن زمانی که دما بیش از حد افزایش می‌یابد بی‌معنی می‌شود، و زمانی که به ۰ نزدیک‌تر می‌شود به متن "چرخه‌ای" سخت‌تولید شده شباهت پیدا می‌کند.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
